TL;DR: After a day fighting with travis, my blog get [auto-deployed](https://github.com/Carreau/posts/blob/master/.travis.yml) with Travis using Nikola !

## A bit of automation

One of the annoying things when I want to post a blog post like this one, is that I never remember hom to deploy my blog. So, why not completly automatize with a script ? 

Well, that one step, but you know what is really good at runnign scripts ? Travis.

Travis have the nice ability to run script in the category `after_success` , or [encrypting file](http://docs.travis-ci.com/user/encrypting-files/), whice allow a nice deployment setup.

## A  description of the process.

The first step is to create an ssh key with empty pass passphrase;
I like to add it (encrypted) to `.travis` folder in my repository.
Travis have nice [doc](http://docs.travis-ci.com/user/encrypting-files/) for that.

Copy the public key to the target github repository deploy key in setting.

In my particular setup the tricky bit where :

To get IPython and nikola master:
  - `- pip install -e git+https://github.com/ipython/ipython.git#egg=IPython`
  - `- pip install -e git+https://github.com/getnikola/nikola.git#egg=nikola`
  
Get the correct layout of folders:
  - blog (gh/carreau/blog)
   - posts (gh/carreau/posts)
   - output (gh/carreau/carreau.github.io)
   
I had to soft link `posts`, as this is the repo which trigger travis build, and is by default cloned into `~/posts` by travis. `carreau/carreau.github.io` is clone via `https` to allow pull request to build (and not push) as the ssh key can only be decrypted on master branch.

Then run `nikola build`.

In the `after_success` step (you might want to run unit-test like non-broken link on your blog) you want to check that you are not in a PR, and  on master branch before trying to decrypt the ssh key and push the built website:


The following snippet works well.
````bash
if [[ $TRAVIS_PULL_REQUEST == false && $TRAVIS_BRANCH == 'master' ]];
```

Travis CL tool gave you the instruction to decrypt the ssh key, you now "just" have to add it 
to the keyring.

```
    eval `ssh-agent -s`
    chmod 600 .travis/travis_rsa
    ssh-add .travis/travis_rsa
    cd ../blog/output
```

And add an ssh remote, which user is **git**:

```
    git remote add ghpages ssh://git@github.com/Carreau/carreau.github.io
```

Push, and you are done (don't forget to commit though) ! 

##  Extra tips

When testing, do not push on master, push on another branch and open the PR manually to see the diff. 
Travis env might defier a bit from yours. 

## Todo

Nikola read metadata file from `.meta` file, which is annoying. I should patch it to read metadata from notebook Metadata. ALso need a JS extension to make that easy.

## Closing thoughts

PR and comments welcommed as usual. 